## Importing Libraries

In [ ]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import pathlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler ,minmax_scale
from keras import layers
from keras import layers
import keras
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Converting Audio Dataset Into Images

In [ ]:
cmap = plt.get_cmap('inferno')
plt.figure(figsize=(8,8))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()


In [ ]:
for g in genres:
    pathlib.Path(f'imgT5_data/{g}').mkdir(parents=True, exist_ok=True)
    for filename in os.listdir(f'./drive/My Drive/genres_original/{g}'):
        songname = f'./drive/My Drive/genres_original/{g}/{filename}'


        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'imgT5_data/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()
        

In [ ]:
header = 'filename chroma_stft spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

## Forming a .CSV File

In [ ]:
import csv
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'./drive/My Drive/Colab Notebooks/GT_DATA/Data_GT/{g}'):
        songname = f'./drive/My Drive/Colab Notebooks/GT_DATA/Data_GT/{g}/{filename}'

        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)

        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())
            

In [ ]:
datanew=pd.read_csv('/content/data2.csv')

In [ ]:
datanew.head()

In [ ]:
datanew = datanew.drop(['filename'],axis=1)

## Label Encoding

In [ ]:
genre_list = datanew.iloc[:, -1]
print(genre_list)
encoder = LabelEncoder()


In [ ]:
y = encoder.fit_transform(genre_list)

## Standardization

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(datanew.iloc[:, :-1], dtype = float))

## Spiliting The Dataset Into Training and Testing

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 , random_state=95)


In [ ]:
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

## Getting Best Possible ANN Model

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV


In [ ]:
def create_model(layers):
    model5=Sequential()
    for i,nodes in enumerate(layers):
        if i==0:
            model5.add(Dense(nodes,activation='relu',input_dim=X_train.shape[1]))
            model5.add(Dropout(0.2))
        else:
            model5.add(Dense(nodes,activation='relu'))
            model5.add(Dropout(0.2))
    model5.add(Dense(10,activation='softmax'))
    model5.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    return model5

In [ ]:
model5=KerasClassifier(build_fn=create_model,verbose=0)

In [ ]:
layers=[(512,256,128,64),(250,120),(250,120,70),(450,200),(250,),(512,)]
param_grid = dict(layers=layers,batch_size=[128],epochs=[80])


In [ ]:
grid1 = GridSearchCV(estimator=model5,param_grid=param_grid)

In [ ]:
grid1_result = grid1.fit(X_train,y_train)

In [ ]:
[grid1_result.best_score_,grid1_result.best_params_]

## Building ANN Model

In [ ]:
from keras import models
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [ ]:
model = models.Sequential()

model.add(layers.Dense(250, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(layers.Dense(120, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(70, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(9, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,
                    y_train,validation_data=(X_test,y_test),
                    epochs=80,
                    batch_size=128)



In [ ]:
model.summary()

In [ ]:
test_loss, test_accuracy = model.evaluate(X_test,y_test)

## Getting Prediction

In [ ]:
pred_y=model.predict(X_test)

In [ ]:
dataf2=pd.DataFrame(pred_y)

In [ ]:
maxValueIndex2 = dataf2.idxmax(axis = 1)

In [ ]:
print(maxValueIndex2)

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix , classification_report
print(confusion_matrix(y_test,maxValueIndex2))

## Classification Report

In [ ]:
print(classification_report(y_test, maxValueIndex2))

## Loss And Accuracy Function Graph

In [ ]:
def plot_history(history):

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="testing accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].legend(loc="upper left")
    axs[0].set_title("Accuracy evaluation")
    fig.tight_layout()

    # create loss sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="testing error")
    axs[1].set_ylabel("loss")
    axs[1].set_xlabel("Epoch")
    axs[1].legend(loc="lower left")
    axs[1].set_title("Loss evaluation")

    plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(X_test,y_test)


In [ ]:
plot_history(history)